## Expectation Maximisation

Expectation maximisation is a really clever technique for estimating the most likely values of some parameters, in a situation where we have both observed and unobserved (hidden) data.

This notebook aims to lay out the mathematics behind the algorithm as clearly as possible.

### Preliminaries

Suppose we have 
- some observed data $z$, 
- unobserved (hidden) data $x$, 
- and parameters $\theta$ we are trying to estimate.

>   I like this labelling of variables because the word "observed" contains a "z" sound. It makes it a bit easier to remember what is what in the equations.

A sensible estimate of the parameters $\hat{\theta}$ would be the parameters that maximise the log likelihood of the observed data $x$.

That is, we would like find the parameters that maximise

$$\log P(z |\ \theta)$$

Note that for any distribution $Q(x)$ on $x$, the unobserved variables, and any particular value of $\theta$, $\theta_p$ say, we have


\begin{align}
& \log P(z |\ \theta_p) \\ \nonumber
& = \log \left( \sum_x P(z, x\ |\ \theta_p) \right) \\
& = \log \left( \sum_x Q(x) \frac{ P(z, x\ |\ \theta_p)}{Q(x)} \right) \\
& \ge \sum_x Q(x) \log \left( \frac{ P(z, x\ |\ \theta_p)}{Q(x)} \right) \\
\end{align}

>   The last inequality here holds via Jensen's inequality, since the penultimate line is the logarithm of an average (where the average is over the distribution of the unobserved variables $Q(x)$). Since log is a concave function, the average of the logarithm'ed quantity is less than or equal to the logarithm of the average of the quantity.

In particular, the above inequality holds when the distribution $Q(x)$ is the distribution of $x$ given the observed data $z$ and *any* parameters $\theta$, $P(x | z, \theta)$.

And in fact, it holds **with equality** in the case when $\theta = \theta_p$, since the expression in the last line above (on the right hand side of the inequality) is

\begin{align*}
& \sum_x P(x | z, \theta_p) \log \left( \frac{ P(z, x|\ \theta_p)}{P(x | z, \theta_p)} \right) \\
& = \sum_x P(x |\ z, \theta_p) \log  P(z|\ \theta_p) \\
& = \log  P(z|\ \theta_p) \sum_x P(x | z, \theta_p) \\
& = \log  P(z|\ \theta_p) 
\end{align*}

>   To move between the first and second lines above, we have simply applied Bayes' theorem: $$P(z, x|\ \theta_p) = P(x | z, \theta_p) P(z | \theta_p)$$

Since the inequality above holds for any particular value of our parameters, $\theta_p$, we can also say that it holds in general for $\log P(x |\ \theta)$ as a function of $\theta$.

If we define a function $f(\theta)$ to be 

$$
f(\theta) = \sum_x Q(x) \log \left( \frac{ P(z, x\ |\ \theta)}{Q(x)} \right)
$$

then that function is a lower bound for the function

$$
\log P(z |\ \theta)
$$

which is the log likelihood we'd like to maximise.

### An iterative update formula for $\theta$

The above all seems a bit abstract right now, but I hope it is fairly clear - we haven't used any advanced mathematics to derive the results above.

Based on what we've shown, we can now define an iterative update procedure for finding a best estimate of $\theta$.

Suppose we have a current set of parameters, our current best guess, $\theta^{(t)}$. We wish to form a better guess from these parameters.

We can do so as follows: let

$$
\theta^{(t + 1)} = \arg\!\max_{\theta}\ g_t(\theta)
$$

where 
$$
g_t(\theta) = \sum_x P(x | z, \theta^{(t)}) \log \left( \frac{ P(z, x |\ \theta)}{P(x | z, \theta^{(t)})} \right) \\
$$

This looks a bit complicated. Let's break it down in words.

- For our next value of $\theta$, we choose the value of $\theta$ which maximises the expression shown for $g_t(\theta)$

- When calculating the expression, the numerator in the fraction depends on $\theta$ (i.e. the parameters we are maximising over) but the other terms use only the current value of $\theta^{(t)}$

>   That second bullet point is quite important. The terms involving $\theta^{(t)}$ don't vary as we vary $\theta$ when we perform the maximisation. They're fixed because they're calculated from $\theta^{(t)}$, our current best guess (and the observed data $z$, and the hidden data $x$ (although the final expression doesn't depend on $x$, because we sum over all possible values of $x$)).

Then from the definition of the update law, we have 

$$ g_t(\theta^{(t)}) \ge g_t(\theta^{(t+1)}) $$

since by definition $\theta^{(t+1)}$ is the value that **maximises** $g_t(\theta)$. 

>   $\theta^{(t)}$ is one possible value of $\theta$, but we maximise over all possible values. So this guarantees that $$ g_t(\theta^{(t)}) \ge g_t(\theta^{(t+1)})$$

Also note that if we put $\theta^{(t)}$ into our function $g$, then we get $\log P(z |\ \theta^{(t)})$.

$$g(\theta^{(t)} = P(z\ |\ \theta^{(t)}$$

Also note that by the argument in the Preliminaries section, $g(\theta)$ is a lower bound on $P(z\ |\ \theta)$. 

$$g(\theta^{(t+1)} \le P(z\ |\ \theta^{(t+1)}$$

So the update rule we have defined allows us to move from a set of parameters $\theta^{(t)}$ to a new set of parameters $\theta^{(t+1)}$.

Using the inequalities above, we have

$$P(z\ |\ \theta^{(t)}) = g_t(\theta^{(t)}) \le g_t(\theta^{(t+1)}) \le P(z\ |\ \theta^{(t+1)})$$

So our new set of parameters $\theta^{(t+1)}$ give us a better log likelihood than the previous set of parameters $\theta^{(t)}$. This is great news! We can keep applying the algorithm recursively and we will reach a maximum of the log-likelihood.

This might, in unfortunate cases, be a *local* maximum rather than a global one. It's hard to solve problems with unobserved data. But this is still a great way to iterate towards a sensible solution, and in many cases the technique shown here will allow us to find a global maximum likelihood estimate for $\theta$.

### A slightly simplified update rule

Note that in the update rule above we are choosing a value of $\theta$ that maximises $g_t(\theta)$. The numerator in the logarithm in the definition of $g_t(\theta)$ allowed us to use the results from the Preliminaries section to prove that the iterative solution will improve our log-likelihood. 

But in fact the numerator doesn't affect the value of $\theta^{(t+1)}$. We can remove it with some algebra, using the properties of logs:

\begin{align*}
g_t(\theta) 
& = \sum_x P(x | z, \theta^{(t)}) \log \left( \frac{ P(z, x |\ \theta)}{P(x | z, \theta^{(t)})} \right) \\
& = \sum_x P(x | z, \theta^{(t)}) \left( \log P(z, x |\ \theta) - \log P(x | z, \theta^{(t)}) \right) \\
& = \sum_x P(x | z, \theta^{(t)}) \log P(z, x |\ \theta) - \sum_x P(x | z, \theta^{(t)}) \log P(x | z, \theta^{(t)}) \\
\end{align*}

Note that the second sum here does not depend on $\theta$, which is the thing we're varying when looking for the argmax, so we can forget about it and use the update rule

$$
\theta^{(t + 1)} = \arg\!\max_{\theta} h_t(\theta)
$$

where 
$$
h_t(\theta) = \sum_x P(x | z, \theta^{(t)}) \log P(z, x |\ \theta)
$$

This expression is actually just an average, or an *expectation*. It is the expectation of the log likelihood of the full data (including both observed variables $z$ and hidden variables $x$) where the average is taken over the distribution of the hidden variables $x$ given the observed variables $z$ and the _current best guess of the parameters_ $\theta^{(t)}$.

So we can now start to understand our update rule a bit better. We can break into two steps:

1) Estimate $P(x |\ z, \theta^{(t)})$
 - $P(x |\ z, \theta^{(t)})$ is the distribution of the hidden variables $x$ given the observed variables $z$ and the current best guess of the parameters $\theta^{(t)}$

2) Maximise the function $h_t(\theta)$ shown above over $\theta$
 - $h_t$ is an expected log-likelihood of the full data ($z$, $x$) over the distribution of the hidden variables found in step 1)
 - this is why the technique is called "Expectation Maximisation"

### Example

Here's a very simple example of the above situation, which will hopefully make the EM algorithm nice and clear.

Suppose we have two coins numbered 0 and 1, each with unknown bias, so the probability of seeing a head when each one is tossed is $p_0$ and $p_1$ respectively.

Suppose we have the results of 5 experiments: in each experiment
- a coin is chosen at random: coin 0 with probability $\lambda$, coin 1 with probability $1 - \lambda$
- the chosen coin is tossed 10 times

We are told the results of the tosses, *but not which coin was tossed in each experiment*.

So our observed data $z$ is the results of all the coin tosses. This can be condensed into the number of heads thrown in each experiment

$$ z = [z_0, z_1, z_2, z_3, z_4], 0 \le z_i \le 10$$

Our hidden data $x$ is which coin was tossed in which experiment

$$ x = [x_0, x_1, x_2, x_3, x_4], x_i \in [0, 1]$$

And our theta is a vector consisting of all the unknown parameters - both coin biases and the probability of choosing coin 0

$$\theta = [p_0, p_1, \lambda]$$

We're going to use the EM algorithm to estimate these parameters. At each stage we'll have a new best guess of the parameters

$$\theta^{(t)} = [p_0^{(t)}, p_1^{(t)}, \lambda^{(t)}]$$

and we'll use the EM algorithm formulas above to make a new guess

$$\theta^{(t+1)} = [p_0^{(t+1)}, p_1^{(t+1)}, \lambda^{(t+1)}]$$

Let's calculate $P(x | z, \theta^{(t)})$. This is the probability of each of our $x_i$, given the observed data and our current best guess of the parameters.

I'm going to stop writing the dependence on $\theta^{(t)}$ explicitly for a moment, to make the notation a bit easier to read. Instead I'll write 

$$P_t( \dots )$$

to mean 

$$P( \dots |\ \theta^{(t)} )$$

We can consider each experiment to be independent, so we have

$$ P(x | z, \theta^{(t)}) = P_t(x | z)  = \prod_{i=1}^5 P_t(x_i | z_i) $$

and

\begin{align*}
P_t(x_i | z_i) = \frac{ P_t(x_i, z_i)}{P_t(z_i)} \\
\end{align*}

by applying Bayes' rule (keeping everything conditional on $\theta^{(t)}$). Each $x_i$ can either be 0 (if coin 0 was chosen for experiment $i$) or 1 (if coin 1 was chosen).

Since we only have two coins, the expression above is simple to break down further. 

\begin{align*}
\frac{ P_t(x_i, z_i)}{P_t(z_i)} 
& = \frac{ P_t(x_i, z_i)}{\sum_{j=0,1} P_t(x_i, z_j)} \\
& = \frac{ 
    P_t(z_i | x_i) 
    P_t( x_i )
    }{
    \sum_{j=0,1}
    P_t(z_i | x_j) 
    P_t(x_j)
} 
\end{align*}

We can now start to write down what these probabilities actually are. We know that, given our current best guess the parameters $\theta^{(t)}$

For any $i$
\begin{align*}
P_t(x_i = 0) & = \lambda^{(t)} \\
P_t(x_i = 1) & = 1 - \lambda^{(t)}
\end{align*}

Each $P_t(z_i | x_j)$ term is simply a likelihood from a binomial distribution, assuming the coin thrown, 0 or 1, is known (since the probability is conditional on $x_j$).

If $x_j = 0$, coin 0 was chosen, and we use $p_0$ in our binomial pmf. If $x_j = 1$, coin 1 was chosen, and we use $p_1$ in our binomial pmf. In either case, we observed $z_i$ heads in experiment $i$, and the total number coin tosses is $n = 10$. So the pmf is

$$
P_t(z_i | x_j) = \binom{10}{z_i} \left(p_{x_j}^{(t)} \right)^{z_i} \left( 1 - p_{x_j}^{(t)} \right)^{10 - z_i}
$$

>    All of the above working is actually a fairly simple application of Bayes' rule, with everything being conditional on the current best guess of the parameters, $\theta^{(t)}$

Putting this all together, we can write down our expression for $P_t(x_i | z_j)$.

\begin{align*}
P_t(x_i = 0 | z_i)
& = \frac{ 
    \binom{10}{z_i} \left(p_0^{(t)} \right)^{z_i} \left( 1 - p_0^{(t)} \right)^{10 - z_i}
    \lambda
    }{
    \lambda
    \binom{10}{z_i} \left(p_0^{(t)} \right)^{z_i} \left( 1 - p_0^{(t)} \right)^{10 - z_i}
    + 
    (1 - \lambda)
    \binom{10}{z_i} \left(p_1^{(t)} \right)^{z_i} \left( 1 - p_1^{(t)} \right)^{10 - z_i}
} 
\end{align*}

and we know $$P_t(x_i = 1 | z_i) = 1 - P_t(x_i = 0 | z_i)$$

>   The big fraction above looks a bit complicated, but you can see the same elements repeated all over the place. In code we can work this out trivially using a function to return the Binomial pmf. Note that all the elements of the equation are known values based on the observed data $z$ and our current best guess of the parameters $\theta^{(t)}$.

In words, $P_t(x_i = 0 | z_i)$ is the probability that the coin used in experiment $i$ is coin 0, based on the observed data (the result of experiment $i$, $z_i$) and the current best guess of the parameters.

We're dealing with a lot of probabilities here, so to try to make the notation a little clearer, let's relabel these probabilities as "weights".

Let's relabel $P_t(x_i = 0 | z_i)$ as $w_{i,0}^{(t)}$ and $P_t(x_i = 0 | z_i)$ as $w_{i,1}^{(t)}$

We now move on to the EM step - we will derive our maximum likelihood estimates of all the parameters, given the function $h_t$ that we will aim to maximise.

$$h_t(\theta) = \sum_x P(x | z, \theta^{(t)}) \log P(x, z |\ \theta)$$


Now we must write down our expression for $\log P(x, z |\ \theta)$.

We'll work in the same was as we did above when deriving the conditional probability $P(x | z, \theta^{(t)}$, applying Bayes' rule to derive a manageable formula for the probability.

Each experiment is independent, so we have a product of the individual likelihoods, which becomes a sum of the individual log likelihoods when we take the logarithm

\begin{align*}
\log P(z, x | \theta) 
& = \log \left( \prod_i P(z_i, \ x_i | \theta) \right) \\
& = \log \left( \prod_i P(z_i |\ x_i, \theta) P(x_i |\ \theta) \right) \\
& = \sum_i \log \left( P(z_i |\ x_i, \theta) P(x_i |\ \theta) \right) \\
\end{align*}

And for each experiment, given $\theta = (p_0, p_1, \lambda)$, we know that 

$$P(x_i = 0) = \lambda$$
$$P(x_i = 1) = 1 - \lambda$$

and 

$$
P(z_i | x_j) = \binom{10}{z_i} p_{x_j}^{z_i} \left( 1 - p_{x_j} \right)^{10 - z_i}
$$

The final expression for $h_t(\theta)$ is

$$h_t(\theta) = \sum_x P(x | z, \theta^{(t)}) \log P(z, x |\ \theta)$$

And consider that the sum over all possible values of $x$ is actually simply the sum over
the two possibilities $x=0$ or $x=1$ for each experiment $i = 1, ... 5$.

\begin{align*}
& = \sum_i w_{i,0}^{(t)} \log \left( \binom{10}{z_i} p_{0}^{z_i} \left( 1 - p_{0} \right)^{10 - z_i} \lambda \right) + 
\sum_i w_{i,1}^{(t)} \log \left( \binom{10}{z_i} p_{1}^{z_i} \left( 1 - p_{1} \right)^{10 z_i} (1 - \lambda) \right) \\
& = \sum_i w_{i,0}^{(t)} \left( \log \binom{10}{z_i} + z_i \log p_{0} + (10 - z_i) \log \left( 1 - p_{0} \right) + \log \lambda \right) \\
& + \sum_i w_{i,1}^{(t)} \left( \log \binom{10}{z_i} + z_i \log p_{1} + (10 - z_i) \log \left( 1 - p_{1} \right) + \log (1 - \lambda) \right) \\
\end{align*}



To find the maximum of this expression w.r.t each of the individual elements of $\theta$, we must differentiate it w.r.t. each variable.

>   In fact, we should partially differentiate the expression w.r.t. each variable, and solve the resulting equations simultaneously, but that won't be necessary here since there are no "cross-terms".

We find

\begin{align*}
\frac{d h_t}{d p_0} =  \sum_i w_{i,0}^{(t)} \left( \frac{z_i}{p_0} + \frac{-(10 - z_i)}{  1 - p_{0} } \right) \\
\end{align*}

This equals 0 when 

\begin{align*}
0 & = \sum_i w_{i,0}^{(t)} \left( \frac{z_i}{p_0} + \frac{-(10 - z_i)}{  1 - p_{0} } \right) \\ \\
\implies \sum_i w_{i,0}^{(t)} \frac{z_i}{p_0} & =  \sum_i w_{i,0}^{(t)} \frac{10 - z_i}{  1 - p_{0} } \\ \\
\implies \frac{1}{p_0} \sum_i w_{i,0}^{(t)} z_i & = \frac{1}{1 - p_0 } \sum_i w_{i,0}^{(t)} (10 - z_i) \\ \\
\implies \frac{p_0}{1 - p_0 }  & = \frac{\sum_i w_{i,0}^{(t)} z_i}{ \sum_i w_{i,0}^{(t)} (10 - z_i) }\\ \\
\implies \frac{1 - p_0 }{p_0}  & = \frac{ \sum_i w_{i,0}^{(t)} (10 - z_i) }{\sum_i w_{i,0}^{(t)} z_i}\\ \\
\implies \frac{1}{p_0} - 1 & = \frac{ \sum_i w_{i,0}^{(t)} (10 - z_i) }{\sum_i w_{i,0}^{(t)} z_i}\\ \\
\implies \frac{1}{p_0} - 1 & = \frac{ \sum_i w_{i,0}^{(t)} 10  }{\sum_i w_{i,0}^{(t)} z_i} - 1 \\ \\
\implies p_0 & = \frac{\sum_i w_{i,0}^{(t)} z_i}{ \sum_i w_{i,0}^{(t)} 10  } \\
\end{align*}

So we see that the EM-algorithm next step estimate for $p_0$ is 

- the weighted sum of all the heads observed across all $i$ experiments (where the weights are the probabilities of each coin flip being coin 0, based on the current best estimate $\theta^{(t)}$) 

divided by

- the weighted sum of all the coin flips across the $i$ experiments (when, once again, the weights are the probabilities of each coin flip being coin 0)

Looking at the above expression for $h_t$, by symmetry we can write down the update rule for $p_1$:

$$
p_1 = \frac{\sum_i w_{i,1}^{(t)} z_i}{ \sum_i w_{i,1}^{(t)} 10  }
$$

Similarly, the value of $\lambda$ that maximises $h_t$ is found by differentiating w.r.t. $\lambda$.

$$\frac{d h_t}{d \lambda} = \sum_i w_{i,0}^{(t)} \frac{1}{ \lambda } + \sum_i w_{i,1}^{(t)} \frac{-1}{ 1 - \lambda } $$

And setting the derivative equal to zero, we find

\begin{align*}
0 & = \sum_i w_{i,0}^{(t)} \frac{1}{ \lambda } + \sum_i w_{i,1}^{(t)} \frac{-1}{ 1 - \lambda } \\ \\
\implies \sum_i w_{i,0}^{(t)} \frac{1}{ \lambda }  & =  \sum_i w_{i,1}^{(t)} \frac{1}{ 1 - \lambda } \\ \\
\implies \frac{ 1 - \lambda }{ \lambda } & = \frac{\sum_i w_{i,1}^{(t)} }{ \sum_i w_{i,0}^{(t)}} \\ \\
\implies  \frac{ 1 }{ \lambda } - 1 & = \frac{\sum_i w_{i,1}^{(t)} }{ \sum_i w_{i,0}^{(t)}} \\ \\
\implies  \frac{ 1 }{ \lambda } & = \frac{\sum_i w_{i,1}^{(t)} }{ \sum_i w_{i,0}^{(t)}} + 1 \\
\implies \frac{ 1 }{ \lambda } & = \frac{\sum_i w_{i,1}^{(t)} + \sum_i w_{i,0}^{(t)}}{ \sum_i w_{i,0}^{(t)}} \\ \\
\implies \lambda & = \frac{ \sum_i w_{i,0}^{(t)}}{\sum_i w_{i,1}^{(t)} + \sum_i w_{i,0}^{(t)}} \\ \\
\implies \lambda & = \frac{ \sum_i w_{i,0}^{(t)}}{ \sum_i 1} \\ \\
\implies \lambda & =\frac{ \sum_i w_{i,0}^{(t)}}{ 5 } \\
\end{align*}

So we see that the best estimate for $\lambda$ at step $t$ is the sum of the weighted probabilities that coin 0 was used, divided by 5 (the number of experiments run). 

I hope you'll agree that this intuitively makes a lot of sense as an update.

### Seeing the formulas above in action

Some of the formulas and derivations above might seem rather long and complicated.

But the approach isn't as hard as it might seem. Often seeing things done in code clarifies the approach. Here we'll implement some functions for applying the formulas above, allowing us to aplpy the EM algorithm in to the coin flip problem described.

In [36]:
from scipy.stats import binom

In [37]:
NUM_EXPERIMENTS = 5
NUM_TOSSES = 10

# The results: the number of heads observed in each experiment
z = [7, 8, 3, 7, 2]

# Our parameters: (p0, p1, lambda)
theta = (0.45, 0.55, 0.5)

# Note that it's important to break symmetry when 
# setting up our initial guesses for parameters. 
#
# If we started with both p0 and p1 as the same
# values, the EM algorithm wouldn't be able to
# split them when applying its formulas (since 
# likelihoods for coin 0 and coin 1 would be 
# identical throughout) and even after updating
# we would never get different p0 and p1 values.
#
# Often in practice we will initialise our guesses
# to random values to avoid this issue.

In [38]:
def binomial_pmf(n, z, p):
    return binom.pmf(z, n, p)

In [39]:
def P_x_i_given_z_i_and_theta(x_i, z_i, theta):
    """ 
    P(x_i | z_i, theta)
    
    This is the probability of the i'th coin 
    being `x_i`, given the observation `z_i` 
    and parameters `theta`.
    
    This is really just a simple application of 
    Bayes' rule.
    """
    assert x_i in (0, 1)  # x_i indicates which coin was used
    
    numer = P_z_i_given_x_i_and_theta(z_i, x_i, theta) * P_x_i_given_theta(x_i, theta)
    denom = (
        P_z_i_given_x_i_and_theta(z_i, 0, theta) * P_x_i_given_theta(0, theta) +
        P_z_i_given_x_i_and_theta(z_i, 1, theta) * P_x_i_given_theta(1, theta)
    )
    p = numer / denom
    
    return p
    
    
def P_z_i_given_x_i_and_theta(z_i, x_i, theta):
    """ 
    P(z_i | x_i, theta)
    
    This is the probability of the i'th coin 
    showing `z_i` heads, given that it is coin 
    `x_i` and parameters `theta`.
    """
    assert x_i in (0, 1)  # x_i indicates which coin was used
    
    # x_i tells us which coin we're using
    # and therefore which probability to
    # pick out from our parameters vector.
    p0, p1, lamb = theta
    p = p0 if x_i == 0 else p1
    
    return binomial_pmf(NUM_TOSSES, z_i, p)
    
    
def P_x_i_given_theta(x_i, theta):
    """ 
    P(x_i | theta)
    
    This is the marginal probability of the i'th
    coin being coin `x_i` (0 or 1), without taking
    into account any observed data. So it is 
    determined by our current lambda value.
    """
    assert x_i in (0, 1)
    
    p0, p1, lamb = theta
    p = lamb if x_i == 0 else 1 - lamb
    
    return p

That's it - that's all the functions we'll need to apply the EM algorithm in this situation.

We just need to implement the update formulas, which are stated in terms of the "weights" $$w_{i,0}^{(t)} = P(x_i = 0 | z_i, \theta^{(t)})$$ and $$w_{i,1}^{(t)} = P(x_i = 1 | z_i, \theta^{(t)})$$

In [40]:
for t in range(10):
    print(f't={t}, theta=({theta[0]:.5f}, {theta[1]:.5f}, {theta[2]:.5f})')
    
    # Step one: calculate our weights.
    # These are based on the observations and the current
    # best estimate of theta.
    
    w_i_0_t = [
        P_x_i_given_z_i_and_theta(0, z[i], theta)
        for i in range(NUM_EXPERIMENTS)
    ]
    w_i_1_t = [
        P_x_i_given_z_i_and_theta(1, z[i], theta)
        for i in range(NUM_EXPERIMENTS)
    ]
    
    # Step two: apply our EM update formulas.
    
    # p0 = P(head when coin 0 is tossed)
    numer = sum(w_i_0_t[i] * z[i] for i in range(NUM_EXPERIMENTS))
    denom = sum(w_i_0_t[i] * NUM_TOSSES for i in range(NUM_EXPERIMENTS))
    new_p0 = numer / denom
    
    # p1 = P(head when coin 1 is tossed)
    numer = sum(w_i_1_t[i] * z[i] for i in range(NUM_EXPERIMENTS))
    denom = sum(w_i_1_t[i] * NUM_TOSSES for i in range(NUM_EXPERIMENTS))
    new_p1 = numer / denom
    
    # lambda = P(coin 0 chosen for each experiment)
    new_lambda = sum(w_i_0_t[i] for i in range(NUM_EXPERIMENTS)) / NUM_EXPERIMENTS
    
    # Update our parameters ready for the next iteration
    theta = (new_p0, new_p1, new_lambda)

print()
print(f't={t+1}, theta=({theta[0]:.5f}, {theta[1]:.5f}, {theta[2]:.5f})')

t=0, theta=(0.45000, 0.55000, 0.50000)
t=1, theta=(0.42385, 0.63970, 0.46189)
t=2, theta=(0.33388, 0.70400, 0.44309)
t=3, theta=(0.27241, 0.72693, 0.41127)
t=4, theta=(0.25603, 0.72888, 0.39945)
t=5, theta=(0.25376, 0.72870, 0.39732)
t=6, theta=(0.25349, 0.72862, 0.39698)
t=7, theta=(0.25345, 0.72860, 0.39693)
t=8, theta=(0.25345, 0.72859, 0.39692)
t=9, theta=(0.25345, 0.72859, 0.39692)

t=10, theta=(0.25345, 0.72859, 0.39692)


### Discussion

If we look at these numbers, they make good intuitive sense.

The algorithm has estimated that 
- coin 0 has $P(\text{head}) = 0.25345$
- coin 1 has $P(\text{head}) = 0.72859$

Looking at our experiments, we have 
- two experiments with a low number of heads, these are probably from coin 0.
- three experiments with a high number of heads, these are probably from coin 1.

So it makes sense that the estimate of $\lambda$ is around 0.4, because it looks like 2 out of our 5 experiments were performed with coin 0.